# DC Location and Transportation Optimization

<img src='https://drive.google.com/uc?id=1dGi9mJwdWCOKMH_8KxwNMzAx4Ch2_B1O' width="600" height="400">  

Data from a manufacturer who distributes nationwide. They are now deciding where to locate their distribution center(s) [DCs]: New York, Atlanta, Chicago, or Los Angeles. Each potential DC location has a fixed annual operating cost if they locate a distribution center there. It also shows the per unit distribution cost from each potential DC location to each region, the capacity of each potential DC location, and the required shipping demand in each region. You need to determine how many and which DC locations to use – and how DCs satisfy the retail city demand needs -- in order to minimize the total fixed operating expenses plus distribution costs.

## Set-up

In [ ]:
# Import libraries
!pip install -q pyomo
from pyomo.environ import *
!apt-get install -y -qq glpk-utils

import pandas as pd
import numpy as np

## Load dataframe

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving P4_Lin.xlsx to P4_Lin (1).xlsx


In [ ]:
dc = pd.read_excel(open('P4_Lin.xlsx','rb'), sheet_name = 'DCdata') 
print(dc.head())
print('DC:', dc.shape)

         location  capacity  fixedcost
0          Durham     20655    3831700
1      Fort Wayne     11279    1848400
2  St. Petersburg     25376    2190000
3          Laredo     40169    1361300
4         Buffalo     25167    1312000
DC: (23, 3)


In [ ]:
# Converting location, capacity, and fixedcost to a list.
dc_loc = dc['location'].tolist()
capacity = dc['capacity'].tolist()
fixedcost = dc['fixedcost'].tolist()

In [ ]:
city = pd.read_excel(open('P4_Lin.xlsx','rb'), sheet_name = 'citydata') 
print(city.head())
print('City:', city.shape)

          city  demand
0     New York    8622
1  Los Angeles    3999
2      Chicago    2716
3      Houston    3284
4      Phoenix    1626
City: (76, 2)


In [ ]:
# Converting demand and names of city to a list.
demand = city['demand'].tolist()
cities = city['city'].tolist()

In [ ]:
dist = pd.read_excel(open('P4_Lin.xlsx','rb'), sheet_name = 'distcost') 
print(dist.head())
print('Dist:', dist.shape)

       Unnamed: 0  New York  Los Angeles  Chicago  Houston  Phoenix  \
0          Durham      2693         1752     1820     2585     2861   
1      Fort Wayne      2690         2581     1851     2175     1851   
2  St. Petersburg      2480         3128     2782     2213     2837   
3          Laredo      5144         5693     3046     5593     4414   
4         Buffalo      3436         4475     4340     5177     4286   

   Philadelphia  San Antonio  San Diego  Dallas  ...  Anchorage  Greensboro  \
0          3446         3814       3100    2098  ...       2240        2169   
1          1895         2529       2525    1872  ...       1042        2098   
2          3252         2588       2735    1111  ...       1521        2142   
3          4492         5348       5757    4681  ...       5324        5901   
4          4186         4459       4863    5337  ...       5642        5111   

   Plano  Newark  Lincoln  Orlando  Irvine  Toledo  Jersey City  Chula Vista  
0   2906    1592   

In [ ]:
# Converting dist cost data to a list of lists
cost = dist.loc[:, 'New York':'Chula Vista'].values.tolist()
cost = np.array(cost)

## Build optimization model

In [ ]:
num_dc = 23 #row
num_city = 76 #column

model = ConcreteModel()

# DVs
model.x = Var(range(num_dc), domain = Binary) # loc dc i, model.x[i]
model.y = Var(range(num_dc), range(num_city), domain = NonNegativeIntegers) # Assign demand to city j to dc i, model.y[i,j]

# Objective: min total cost
var_cost = sum(model.y[i,j]*cost[i][j] for i in range(num_dc) for j in range(num_city))
fix_cost = sum(model.x[i]*fixedcost[i] for i in range(num_dc))
model.Objective = Objective(expr = var_cost + fix_cost, sense = minimize)

# Constraint: capacity
model.capacity = ConstraintList()
for i in range(num_dc):
  model.capacity.add(sum(model.y[i,j] for j in range (num_city)) <= capacity[i]*model.x[i])

# Contraint: demand
model.demand = ConstraintList()
for j in range(num_city): 
  model.demand.add(sum(model.y[i,j] for i in range(num_dc)) == demand[j])

In [ ]:
# Solve the model
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 # Specifies the time limit (in seconds)
opt.options['mipgap'] = .0 # Specifies the optimality gap tolerance (.01 means can stop if <1% of optimal obj)
results = opt.solve(model) # Set tee=True can see the details.

In [ ]:
# Print the results
print('Total Cost: ', model.Objective())

Total Cost:  58006498.0


In [ ]:
print('DC locations:')
for i in range(num_dc):
    if model.x[i]() > 0:
        print(dc_loc[i],', ', 'Fixed Cost:', fixedcost[i])

DC locations:
Lubbock ,  Fixed Cost: 1323800
Glendale ,  Fixed Cost: 1303100
Winston–Salem ,  Fixed Cost: 2856900
Irving ,  Fixed Cost: 1275700
Fremont ,  Fixed Cost: 2114800


In [ ]:
print('City Distribution:')
for i in range(num_dc):
    if model.x[i]() > 0:
        for j in range(num_city):
            if model.y[i,j]() > 0:
                print('DC:', dc_loc[i], ',', 'City:', cities[j], ',', model.y[i,j](), 'thousand units,', 'Cost:', model.y[i,j]()*dist.loc[i][j+1])

City Distribution:
DC: Lubbock , City: Los Angeles , 3999.0 thousand units, Cost: 3175206.0
DC: Lubbock , City: Houston , 3284.0 thousand units, Cost: 3277432.0
DC: Lubbock , City: San Diego , 1419.0 thousand units, Cost: 188727.0
DC: Lubbock , City: San Jose , 1035.0 thousand units, Cost: 928395.0
DC: Lubbock , City: Indianapolis , 863.0 thousand units, Cost: 139806.0
DC: Lubbock , City: Seattle , 724.0 thousand units, Cost: 424988.0
DC: Lubbock , City: Washington, D.C. , 693.0 thousand units, Cost: 314622.0
DC: Lubbock , City: Memphis , 620.0 thousand units, Cost: 344720.0
DC: Lubbock , City: Las Vegas , 672.0 thousand units, Cost: 213696.0
DC: Lubbock , City: Albuquerque , 416.0 thousand units, Cost: 242112.0
DC: Lubbock , City: Tucson , 768.0 thousand units, Cost: 148224.0
DC: Lubbock , City: Mesa , 614.0 thousand units, Cost: 140606.0
DC: Lubbock , City: Kansas City , 742.0 thousand units, Cost: 1136002.0
DC: Lubbock , City: Long Beach , 553.0 thousand units, Cost: 376040.0
DC: Lu